In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# 1. Define Network Models

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(N_STATES,50)
        self.fc1.weight.data.normal_(0,0.5)
        self.fc2 = nn.Linear(50,N_ACTIONS)
        self.fc2.weight.data.normal_(0,0.5)
    def forward(self,x):
        x=self.fc1(x)
        x=F.relu(x)
        action = self.fc2(x)
        return action

In [6]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE = 100   # target update frequency
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

[2018-06-25 22:07:59,773] Making new env: CartPole-v0


# 2. Create DQN Networks

In [9]:
class DQN(object):
    
    def __init__(self):
        self.eval_net, self.target_net = Net(),Net()
        self.target_update_counter = 0
        self.memory_counter = 0
        self.memory = np.zeros((MEMORY_CAPACITY,N_STATES*2+2))
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(),lr=LR)
        self.loss_func = nn.MSELoss()
        
    def choose_action(self,x):
        x = torch.unsqueeze(torch.FloatTensor(x),0)
        # Input only one sample
        if np.random.uniform()<EPSILON:
            action_value = self.eval_net(x)
            action = torch.max(action_value,1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # Double Check
        else:
            action = np.random.randint(0,N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE) # Double Check
        return action
    
    def store_transition(self,s,a,r,s_):
        transition = np.hstack((s,[a,r],s_)) # Double check 
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index,:]=transition
        self.memory_counter+=1
        
    def learn(self):
        if self.target_update_counter % TARGET_REPLACE == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.target_update_counter += 1
        
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])
        
        # Do the evaluation 
        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [10]:
dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_


('Ep: ', 175, '| Ep_r: ', 1.48)
('Ep: ', 176, '| Ep_r: ', 4.15)
('Ep: ', 177, '| Ep_r: ', 1.76)
('Ep: ', 178, '| Ep_r: ', 1.39)
('Ep: ', 179, '| Ep_r: ', 3.44)
('Ep: ', 180, '| Ep_r: ', 4.59)
('Ep: ', 181, '| Ep_r: ', 10.22)
('Ep: ', 182, '| Ep_r: ', 2.15)
('Ep: ', 183, '| Ep_r: ', 5.33)
('Ep: ', 184, '| Ep_r: ', 2.73)
('Ep: ', 185, '| Ep_r: ', 2.37)
('Ep: ', 186, '| Ep_r: ', 3.58)
('Ep: ', 187, '| Ep_r: ', 2.71)
('Ep: ', 188, '| Ep_r: ', 2.84)
('Ep: ', 189, '| Ep_r: ', 1.21)
('Ep: ', 190, '| Ep_r: ', 2.83)
('Ep: ', 191, '| Ep_r: ', 4.84)
('Ep: ', 192, '| Ep_r: ', 5.82)
('Ep: ', 193, '| Ep_r: ', 2.33)
('Ep: ', 194, '| Ep_r: ', 2.56)
('Ep: ', 195, '| Ep_r: ', 1.27)
('Ep: ', 196, '| Ep_r: ', 1.62)
('Ep: ', 197, '| Ep_r: ', 2.57)
('Ep: ', 198, '| Ep_r: ', 3.04)
('Ep: ', 199, '| Ep_r: ', 1.8)
('Ep: ', 200, '| Ep_r: ', 1.31)
('Ep: ', 201, '| Ep_r: ', 3.17)
('Ep: ', 202, '| Ep_r: ', 3.3)
('Ep: ', 203, '| Ep_r: ', 2.03)
('Ep: ', 204, '| Ep_r: ', 1.98)
('Ep: ', 205, '| Ep_r: ', 2.8)
('Ep: ', 

In [16]:
s = env.reset()

In [17]:
dqn.choose_action(s)

1

In [18]:
s_, r, done, info = env.step(a)

In [33]:
??env.step()

In [21]:
s

array([-0.02242197, -0.02170639, -0.02709821, -0.02154262])

In [22]:
x, x_dot, theta, theta_dot = s_

In [23]:
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2